Загрузка данных из csv, их предобработка и запуск классификаторов
=======

# Введение

В этом примере мы буде работать с датасетом, собранным по результатам опроса молодых людей, изучающих статистику на Фукультете социальных и экономических наук Коменского университета Братиславы. Датасет содержит ответы на вопросы, которые можно разделить на следующие категории:

- Предпочтения в музыке (19 пунктов)
- Предпочтения в жанрах кино (12 пунктов)
- Хобби и интересы (32 пунктов)
- Фобии и страхи (10 пунктов)
- Привычки к здоровому и нездоровому образу жизни (3 пунктов)
- Личные качества, взгляды на жизнь (57 пунктов)
- Основные денежные траты (7 пунктов)
- Демографические данные (10 пунктов)

Сперва мы загрузим датасет, заменим некоторые строковые признаки на числовые для более удобной работы. Затем, выберем набор признаков, с которым будем работать. Не обязательно использовать все данные. В качестве пробной задачи постараемся научиться предсказывать пол респондента по его предпочтениям, мнениям и фобиям, не используя такие прямые физиологические данные, как вес и рост.

Датасет хранится в файле `responses.csv`. Любой анализ данных начинается с понимания того, с чем предстоит работать. Чтобы ознакомиться с датасетом, его можно открыть в табличном процессоре Libre Office Calc (или, на худой конец, в MS Excel).

Большинство столбцов содержат отношение респондента к тому или иному явлению по шкале от 1 до 5. Например, графа `Chemistry` означает степень инетереса участника опроса химией, где 1 - совсем не интересно, 5 - увлекаюсь этой наукой. Некоторые содержат категориальные данные, некоторые из них - упорядочены (например, частота употребления алкоголя в графе `Alcohol`).

Мы обычим бинарный классификатор предсказывать пол респондента (`Gender`) по его предпочтениям в музыке и кино, а также фобиям, используя разные алгоритмы классификации.

# Загрузка данных
Начнём с загрузки данных. Нам понадобится библиотека `pandas`, в которой есть класс `DataFrame`, который умеет хранить таблицу и работать с ними. Это своего рода аналог табличного процессора Calc или Excel, с которым можно работать из кода на Python. Также, нам понадобится библиотека `numpy`. Она используется для быстрой и эффективной работы с многомерными массивами чисел, векторами и матрицами. Модуль `numpy` является стандартом для решения таких задач на Python.


In [36]:
import pandas as pd
import numpy as np

Теперь, загрузим датасет и проверим, что всё загрузилось правильно:

In [37]:
data_frame = pd.read_csv("responses.csv")
data_frame

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,Age,Height,Weight,Number of siblings,Gender,Left - right handed,Education,Only child,Village - town,House - block of flats
0,5.0,3.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,...,20.0,163.0,48.0,1.0,female,right handed,college/bachelor degree,no,village,block of flats
1,4.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0,...,19.0,163.0,58.0,2.0,female,right handed,college/bachelor degree,no,city,block of flats
2,5.0,5.0,2.0,2.0,3.0,4.0,5.0,3.0,5.0,3.0,...,20.0,176.0,67.0,2.0,female,right handed,secondary school,no,city,block of flats
3,5.0,3.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,...,22.0,172.0,59.0,1.0,female,right handed,college/bachelor degree,yes,city,house/bungalow
4,5.0,3.0,4.0,3.0,2.0,4.0,3.0,5.0,3.0,1.0,...,20.0,170.0,59.0,1.0,female,right handed,secondary school,no,village,house/bungalow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,5.0,2.0,5.0,2.0,2.0,5.0,4.0,4.0,4.0,3.0,...,20.0,164.0,57.0,1.0,female,right handed,secondary school,no,city,house/bungalow
1006,4.0,4.0,5.0,1.0,3.0,4.0,1.0,4.0,1.0,1.0,...,27.0,183.0,80.0,5.0,male,left handed,masters degree,no,village,house/bungalow
1007,4.0,3.0,1.0,1.0,2.0,2.0,2.0,3.0,4.0,1.0,...,18.0,173.0,75.0,0.0,female,right handed,secondary school,yes,city,block of flats
1008,5.0,3.0,3.0,3.0,1.0,3.0,1.0,3.0,4.0,1.0,...,25.0,173.0,58.0,1.0,female,right handed,college/bachelor degree,no,city,block of flats


Теперь, можно немного попрактиковаться в работе с `pandas`. Например, посмотрим, какие значения в принципе может содержать столбец `Alcohol`. Для этого обратимся к столбцу при помощи `data_frame["Alcohol"]`:

In [38]:
print(data_frame["Alcohol"].unique())

['drink a lot' 'social drinker' 'never' nan]


Обратите внимание на поле `nan`. Вообще говоря, это расшифровывается, как "not a number", но в данном случае имеет более широкий смысл, и означает, что значение не предоставлено. То есть для некоторых строк таблицы в столбце `Alcohol` пусто.

Теперь, заготовим списки столбцов, которые будут нас интересовать в дальнейшем. Это пригодится нам, чтобы отделить только нужные признаки от общей массы:

In [39]:
personal = ["Age","Height","Weight","Number of siblings"]
music_preferences = ["Pop","Rock","Metal or Hardrock","Punk","Hiphop, Rap","Reggae, Ska","Swing, Jazz","Rock n roll","Alternative","Latino","Techno, Trance","Opera"]
cinema_preferences = ["Movies","Horror","Thriller","Comedy","Romantic","Sci-fi","War","Fantasy/Fairy tales","Animated","Documentary","Western","Action"]
spending = ["Spending on looks","Spending on gadgets","Spending on healthy eating"]
phobias = ["Flying","Storm","Darkness","Heights","Spiders","Snakes","Rats","Ageing","Dangerous dogs","Fear of public speaking"]
interests = ["History","Psychology","Politics","Mathematics","Physics","Internet","PC","Economy Management","Biology","Chemistry","Reading","Geography","Foreign languages","Medicine","Law"]


Соберём интересущие нас признаки в один большой список. Обратите внимание, в Python операция `+` объединяет списки. Ещё, выберем целевой признак, тот, который мы будем предсказывать:

In [40]:
feature_columns = cinema_preferences + music_preferences + phobias
target_feature = "Gender"

Выберем интересующие нас признаки из датасета, включая тот, который мы хотим предсказать. Попутно удалим строки, содержащие хотя бы один `nan`.


In [41]:
partial_data = data_frame[feature_columns + [target_feature]].dropna()
lines_count = len(partial_data)

На всякий случай, выполним рандомизацию датасета. Для этого существует функция `sample`. Чтобы была воспроизводимость результатов, зададим параметр `random_state=1234` - инициализация генератора случайных чисел каким-то фиксированным значением

In [42]:
partial_data = partial_data.sample(frac=1.0, random_state=1234)

Теперь, преобразуем `partial_data` в матрицу признаков и вектор разметки:

In [43]:
X = np.array(partial_data[feature_columns].values)
y = np.array(partial_data[target_feature].values)

Теперт X и y имеют тип `numpy.array`. Разделим весь датасет на обучающую и валидационную выборки:

In [44]:
train_part = 0.8
train_lines = int(lines_count * train_part)

X_train = X[:train_lines, :]
y_train = y[:train_lines]

X_val = X[train_lines:, :]
y_val = y[train_lines:]

Обратите внимание на выражение с двоеточиями. Запись `X[from:to]` означает, что из массива `X` нужно взять элементы начиная с номера `from` и заканчивая номером `to-1`. Если на месте `from` ничего не указано: `X[:to]`, то берутся элементы, начиная с нулевого и заканчивая `to-1`. Аналогично, запись`X[from:]` означает "взять все элементы, начиная с `from` и до конца массива.

# Обучение классификатора

Теперь всё готово для обучения и запуска модели. Для экспериментов, импортируем из пакета `sklearn` несколько бинарных классификаторов, а также функции для подсчёта метрик:

In [45]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import scale
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

Создадим объект-классификатор. Поэкспериментируйте с разными классификаторами:

In [46]:
# Решающее дерево
# clf = DecisionTreeClassifier()

# Метод k ближайших сосоедей
# clf = KNeighborsClassifier(n_neighbors=10)

# Случайный лес
# clf = RandomForestClassifier()

# Метод опорных векторов
# clf = SVC()

# Логистическая регрессия
clf = LogisticRegression()

Запустим обучение классификатора на выборке `X_train` и `y_train`. Оно вызывается одинаково для всех представленных выше алгоритмов. Стоит заметить, что классификаторы их библиотеки `sklearn` могут работать не только с классами в виде чисел 1 и 0, им подходят в том числе строковые называния классов. Внутри, эти классификаторы сами сделают всю нужную работу, и сопоставят строкам числа.

In [47]:
clf.fit(X_train, y_train)

LogisticRegression()

Теперь, необходимо выполнить валидацию. Запустим предсказание на валидационной выборке `X_val`:

In [48]:
y_predict = clf.predict(X_val)

Теперь, у нас есть наборы истинных и предсказанных классов для валидационной выборки `y_val` и `y_predict`. Мы можем использовать их для вычисления метрик, таких, как precision, recall и accuracy. Для этого можно использовать готовую функцию `classification_report`:

In [49]:
print(classification_report(y_val, y_predict))

              precision    recall  f1-score   support

      female       0.87      0.84      0.85       101
        male       0.81      0.84      0.83        82

    accuracy                           0.84       183
   macro avg       0.84      0.84      0.84       183
weighted avg       0.84      0.84      0.84       183



По данным метрикам можно сделать выводы о работе классификатора. Чтобы непосредственно посмотреть количество верных ошибочных предсказаний, можно использовать функцию `confusion_matrix`. Она выдаёт матрицу, в которой строки соответствуют реальным классам, столбцы - предсказанным, а значения в соответствующих строках и столбцах, соответственно, количеству случаев:

In [50]:
print(confusion_matrix(y_val, y_predict))

[[85 16]
 [13 69]]


Чем больше диагональные элементы по сравнению с недиагональными, тем лучше работает классификатор. В данном случае из всех `female` корректно были предсказаны 85, а 16 классифицированы, как `male`. Из всех `male` 69 определены корректно, а 13 определены, как `female`.

# Задание

1. Используя предоставленные примеры кода, определите, какой тип классификатора наиболее хорошо справляется с задачей классификации мужчин и женщин по результатам опроса. Для метода k ближайших соседей определите оптимальное количество ближайших соседей.
2. Для наилучшего классификатора экспериментально определите, какие категории признаков наиболее полезны при классификации: `personal`, `music_preferences`, `cinema_preferences`, `spending`, `phobias`, `interests`.